<a href="https://colab.research.google.com/github/LucasMatuszewski/Python-colab-notebooks/blob/main/notebooks/aMUSEd_Image_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# aMUSEd Image generation - 512px and 256px models
Based on: [aMUSEd: An Open MUSE Reproduction](https://huggingface.co/amused/amused-512) (which is based on Google MUSE)

In [21]:
%%javascript
var codeCells = document.querySelectorAll("div.view-lines");
console.log(codeCells)
for (var i = 0; i < codeCells.length; i++) {
    var lines = codeCells[i].querySelectorAll('.view-line').length;
    console.log(lines)
    var numbers = Array(lines).fill(0).map((_, i) => i + 1).join("\n");
    var lineNumbersDiv = document.createElement('div');
    lineNumbersDiv.textContent = numbers;
    codeCells[i].parentElement.insertBefore(lineNumbersDiv, codeCells[i]);
}


<IPython.core.display.Javascript object>

# import dependencies and install diffusers

In [1]:
!pip install -q diffusers
!pip install transformers
import torch
from diffusers import AmusedPipeline, AmusedImg2ImgPipeline, AmusedInpaintPipeline
# Available pipelines:
# AmusedPipeline - base text 2 image
# AmusedImg2ImgPipeline - image + text 2 image
# AmusedInpaintPipeline - image + mask + text 2 image

Defaulting to user installation because normal site-packages is not writeable


/home/lucas/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
/home/lucas/.local/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Load aMUSEd 512px or 256px model (with optional acceleration for faster restults)

In [31]:
# Colab's @param doesn't work in Jupyter, to get similar UX use Jupyter Widgets:
# https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
model_name = "amused/amused-512"  # @param ["amused/amused-512", "amused/amused-256"] {type:"string"}
mode = "img"  # @param ["txt", "img", "img_mask"] {type:"string"}
should_accelerate = True # @param {type: "boolean"}
size = 512 if model_name == "amused/amused-512" else 256
# width = size # @param {type: "string"}
# height = size # @param {type: "string"}

if mode != "txt":
    from diffusers.utils import load_image

# Load the model first
try:
    print("Loading the model...")
    Pipeline = AmusedInpaintPipeline if mode == "img_mask" \
               else AmusedImg2ImgPipeline if mode == "img" \
               else AmusedPipeline
    pipe = Pipeline.from_pretrained(
        model_name, variant="fp16", torch_dtype=torch.float16
    )
except Exception as e:
    raise RuntimeError(f"Couldn't Load the model: {e}")

# Is GPU available?
if not torch.cuda.is_available():
    print("\nRunning on CPU !! Go to 'Runtime' manu and 'Change runtime type' to 'GPU'")
    raise RuntimeError("GPU not found. Please switch to a GPU runtime.")

generator = torch.Generator('cuda').manual_seed(0)

# Accelerate model and generator
if should_accelerate:
    try:
        !pip install accelerate
        from accelerate import Accelerator
        accelerate_available = True
    except ImportError as e:
        print(f"Couldn't import Accelerator: {e}")
        accelerate_available = False
    if accelerate_available:
        accelerator = Accelerator(fp16=True)  # Or with other desired settings
        pipe, generator = accelerator.prepare(pipe, generator)

# Send the model to the GPU
pipe = pipe.to("cuda")



Loading the model...


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Loading pipeline components...: 100%|█████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.02it/s]


Defaulting to user installation because normal site-packages is not writeable


TypeError: Accelerator.__init__() got an unexpected keyword argument 'fp16'

# Image with Mask and prompt

In [28]:
# define image loading function:
def image_loader(image_url, color=True):
    return load_image(image_url) \
        .resize((size, size)) \
        .convert("RGB" if color else "L") # L = grayscale (Luminance)

# define a function for later usage:
def generate_image(prompt, input_image_url=None, mask_url=None, strength=0.7):
    try:
        input_image = image_loader(input_image_url) if input_image_url else None
        mask = image_loader(mask_url, color=False) if mask_url else None

        # To return multiple images:
        # number_of_images = 1 # @param {type:"slider", min:1, max:20, step:1}
        # for seed in range(number_of_images):
        #     print(f"Generating image {seed}...")
        #     image = pipe(prompt, input_image, mask, generator=torch.Generator('cuda').manual_seed(seed)).images[0]
        #     # image.save(f'inpainting_256_{seed}.png')
        #     print(f"Image {seed} generated!")
        print("Generating image...")
        image = pipe(prompt, input_image, mask, generator).images[0]
        print("Image generated!")
        return image

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

prompt = "a man with glasses"  # @param {type: "string"}
input_image_url = "https://raw.githubusercontent.com/huggingface/amused/main/assets/inpainting_256_orig.png" # @param {type: "string"}
mask_url = "https://raw.githubusercontent.com/huggingface/amused/main/assets/inpainting_512_mask.png"  # @param {type: "string"}
strength = 0.7 # @param {type:"slider", min:0, max:1, step:0.1}

generate_image(
      prompt,
      input_image_url = input_image_url if mode != "txt" else None,
      mask_url = mask_url if mode == "img_mask" else None,
      strength = strength
  )

# If you want to save the image to a file:
# image = pipe(prompt, input_image, mask, generator=torch.Generator('cuda').manual_seed(0)).images[0]
# image.save('inpainting_512.png')


Generating image...
An error occurred: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 30.50 MiB is free. Including non-PyTorch memory, this process has 3.20 GiB memory in use. Of the allocated memory 3.12 GiB is allocated by PyTorch, and 9.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
